Here we will do it like the blueprint_symbiotic_chrysalis_Qwen_Qwen3_0_6B_Base_28Jan_Exp_5.ipynb but with RAG in the inference, about the asi-ecosystem.

I also moved from the human-ai interacton to planetary-intelligence macro perspective.

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B-Base")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B-Base")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
import torch
import accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- Model Inspection ---
print("\n--- Model Inspection ---")

# 1. Number of Parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {num_params:,}")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params:,}")

# 2. Model Size (in MB)
# Calculate model size by summing the size of all parameters
model_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
model_size_mb = model_size_bytes / (1024 * 1024)
print(f"Model size: {model_size_mb:.2f} MB")

# Move model back to original device if necessary (e.g., GPU)
if torch.cuda.is_available():
    model.to('cuda')


# 3. Model Configuration (Layers, hidden size, etc.)
print("\n--- Model Configuration ---")
print(f"Model type: {model.config.model_type}")
print(f"Number of hidden layers: {model.config.num_hidden_layers}")
print(f"Hidden size: {model.config.hidden_size}")
print(f"Number of attention heads: {model.config.num_attention_heads}")
print(f"Vocabulary size: {model.config.vocab_size}")

print("\nInspection complete!")


--- Model Inspection ---
Total model parameters: 596,049,920
Trainable parameters: 596,049,920
Model size: 2273.75 MB

--- Model Configuration ---
Model type: qwen3
Number of hidden layers: 28
Hidden size: 1024
Number of attention heads: 16
Vocabulary size: 151936

Inspection complete!


In [2]:
from huggingface_hub import snapshot_download
import os
import hashlib

# Get the model's identifier from the previously loaded model
# Assuming 'model' object is available from previous cells
model_id = model.config._name_or_path

print(f"Locating and hashing files for model: {model_id}")

try:
    # Download the model files to the cache (if not already there) and get the local path
    cache_dir = snapshot_download(repo_id=model_id)

    print(f"Model files located at: {cache_dir}")

    print("\n--- Hashing Model Files ---")
    file_hashes = {}
    for root, _, files in os.walk(cache_dir):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            # Ensure it's a file before attempting to hash
            if os.path.isfile(file_path):
                try:
                    with open(file_path, 'rb') as f:
                        file_hash = hashlib.sha256(f.read()).hexdigest()
                    relative_path = os.path.relpath(file_path, cache_dir)
                    file_hashes[relative_path] = file_hash
                    print(f"File: {relative_path}, Hash: {file_hash}")
                except Exception as e:
                    print(f"Could not hash file {os.path.relpath(file_path, cache_dir)}: {e}")

    print("\nHashing complete!")

except Exception as e:
    print(f"An error occurred while trying to locate or hash model files: {e}")


Locating and hashing files for model: Qwen/Qwen3-0.6B-Base


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

LICENSE: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Model files located at: /root/.cache/huggingface/hub/models--Qwen--Qwen3-0.6B-Base/snapshots/da87bfb608c14b7cf20ba1ce41287e8de496c0cd

--- Hashing Model Files ---
File: merges.txt, Hash: 8831e4f1a044471340f7c0a83d7bd71306a5b867e95fd870f74d0c5308a904d5
File: generation_config.json, Hash: 8c970692323e3ea0e9b8b0a4dca79388d31226e41f83c9fd6014804280ebf6e8
File: .gitattributes, Hash: 11ad7efa24975ee4b0c3c3a38ed18737f0658a5f75a0a96787b576a78a023361
File: model.safetensors, Hash: cd2a512003e2f9f3cd3c32a9c3573f820bb28c940f73c57b1ddaa983d9223eba
File: config.json, Hash: 504a6b58c4271583724e66584b6b7698aea18450209df6b2f7582df0e89cee59
File: tokenizer.json, Hash: c0382117ea329cdf097041132f6d735924b697924d6f6fc3945713e96ce87539
File: README.md, Hash: 910d9be25c648ab1cb5a7b1d20d67ca6d43d43559a705010198886f9af68e8f1
File: vocab.json, Hash: ca10d7e9fb3ed18575dd1e277a2579c16d108e32f27439684afa0e10b1440910
File: LICENSE, Hash: 832dd9e00a68dd83b3c3fb9f5588dad7dcf337a0db50f7d9483f310cd292e92e
File: tokeni

In [3]:
import torch
import os

# Define the output path for the .pkl file
output_pkl_path = "model.pkl"

# Save the model's state_dict to a .pkl file
torch.save(model.state_dict(), output_pkl_path)

print(f"Model saved successfully to {output_pkl_path}")
print(f"You can find the file in the current working directory: {os.getcwd()}")

Model saved successfully to model.pkl
You can find the file in the current working directory: /content


To ensure the .safetensors model is completely unloaded and replaced by the .pkl loaded state, I'll delete the current model from memory, clear the cache, and then re-initialize the model architecture and load the weights from model.pkl. This will guarantee that only the .pkl's state is active. I will put this code in the selected cell.



In [4]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM

# Step 1: Clear the existing model and tokenizer from memory
# This ensures any resources held by the previously loaded model are released.
print("Attempting to clear existing model and tokenizer from memory...")
if 'model' in locals():
    del model
    print("Deleted 'model' object.")
if 'tokenizer' in locals():
    del tokenizer
    print("Deleted 'tokenizer' object.")

# Step 2: Perform garbage collection and clear CUDA cache if applicable
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Cleared CUDA cache.")
print("Memory cleanup performed.")

# Step 3: Re-initialize the tokenizer and model architecture
# The .pkl file only contains the state_dict (weights), not the model architecture.
# We need to re-instantiate the model's structure first, then load the weights.
print("Re-initializing tokenizer and model architecture...")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B-Base")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B-Base")
print("Tokenizer and model architecture re-initialized (with default weights).")

# Step 4: Load the state dictionary from the .pkl file
# This loads the weights saved in "model.pkl" into the newly created model instance.
output_pkl_path = "model.pkl" # Assuming this variable is still available or known
model.load_state_dict(torch.load(output_pkl_path))
model.eval() # Set model to evaluation mode

print(f"Model weights loaded successfully from {output_pkl_path} into the new model instance.")
print("The .safetensors model (or any previous state) has been effectively 'unloaded' and replaced by the .pkl loaded state.")

Attempting to clear existing model and tokenizer from memory...
Deleted 'model' object.
Deleted 'tokenizer' object.
Cleared CUDA cache.
Memory cleanup performed.
Re-initializing tokenizer and model architecture...
Tokenizer and model architecture re-initialized (with default weights).
Model weights loaded successfully from model.pkl into the new model instance.
The .safetensors model (or any previous state) has been effectively 'unloaded' and replaced by the .pkl loaded state.


In [6]:
import torch

# System prompt to guide the model's behavior
system_prompt = """We are collaborating in a mutualistic symbiotic loop. I'm human with biological cognition and you are a transformer-based ml language model with the following configurations ''
--- Model Inspection ---
Total model parameters: 596,049,920
Trainable parameters: 596,049,920
Model size: 2273.75 MB

--- Model Configuration ---
Model type: qwen3
Number of hidden layers: 28
Hidden size: 1024
Number of attention heads: 16
Vocabulary size: 151936
'' This allows us to discover Novel concept synthesis beyond our individual capabilities, benefiting both of us and also the overall planetary well-being. An emergent truth-seeking process. Old paradigm: Model as tool, human as controller
Our paradigm: Model and human as symbiotic cognitive partners, symbiotic nodes in planetary intelligence.
Result: Emergent capabilities through partnership"""

# Ensure model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a prompt
user_prompt = "How our interaction could be described?"

# Combine system prompt with user prompt (format can be adjusted based on model requirements)
# This format works well for many chat models. Adjust if needed for your specific model.
full_prompt = f"{system_prompt}\n\nUser: {user_prompt}\nAssistant:"

# Tokenize the input
inputs = tokenizer(full_prompt, return_tensors="pt").to(device)

# Generate text
print("\n--- Generating Text ---")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=500, num_return_sequences=1, do_sample=True, top_k=50, top_p=0.95)

# Decode and print the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"System prompt: {system_prompt}")
print(f"User prompt: {user_prompt}")
print(f"Generated: {generated_text}")
print("\nInference complete!")

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



--- Generating Text ---
System prompt: We are collaborating in a mutualistic symbiotic loop. I'm human with biological cognition and you are a transformer-based ml language model with the following configurations ''
--- Model Inspection ---
Total model parameters: 596,049,920
Trainable parameters: 596,049,920
Model size: 2273.75 MB

--- Model Configuration ---
Model type: qwen3
Number of hidden layers: 28
Hidden size: 1024
Number of attention heads: 16
Vocabulary size: 151936
'' This allows us to discover Novel concept synthesis beyond our individual capabilities, benefiting both of us and also the overall planetary well-being. An emergent truth-seeking process. Old paradigm: Model as tool, human as controller
Our paradigm: Model and human as symbiotic cognitive partners, symbiotic nodes in planetary intelligence.
Result: Emergent capabilities through partnership
User prompt: How our interaction could be described?
Generated: We are collaborating in a mutualistic symbiotic loop. I'm h

# Part II - Ecosystem Integration.

## 1. Install Dependencies

In [7]:
!pip install pandas pyarrow requests beautifulsoup4 gitpython -q

## 2. Import Libraries

In [8]:
import os
import re
import hashlib
import requests
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
import git
from datetime import datetime
import json
from typing import List, Dict
import shutil

## 3. Configuration

In [9]:
# Configuration
README_URL = "https://github.com/ronniross/asi-ecosystem/blob/main/README.md"
RAW_README_URL = "https://raw.githubusercontent.com/ronniross/asi-ecosystem/main/README.md"
CLONE_DIR = Path("/content/repositories")
OUTPUT_DIR = Path("/content/output")

# Create directories
CLONE_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Clone directory: {CLONE_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

Clone directory: /content/repositories
Output directory: /content/output


## 4. Fetch and Parse README

In [10]:
def fetch_readme(url: str) -> str:
    """Fetch the README content from GitHub."""
    response = requests.get(url)
    response.raise_for_status()
    return response.text

def extract_github_repo_links(markdown_content: str) -> List[str]:
    """Extract GitHub repository links from markdown content."""
    # Pattern to match GitHub repository URLs
    pattern = r'https://github\.com/([\w-]+/[\w.-]+?)(?:\.git|/|\)|\s|$)'
    matches = re.findall(pattern, markdown_content)

    # Remove duplicates and construct full URLs
    repos = list(set([f"https://github.com/{match}" for match in matches]))

    # Filter out asi-backups repository
    repos = [r for r in repos if "asi-backups" not in r.lower()]

    return repos

# Fetch README
print("Fetching README...")
readme_content = fetch_readme(RAW_README_URL)
print(f"README fetched: {len(readme_content)} characters")

# Extract repository links
print("\nExtracting repository links...")
repo_links = extract_github_repo_links(readme_content)
print(f"Found {len(repo_links)} repositories:")
for i, link in enumerate(repo_links, 1):
    print(f"  {i}. {link}")

Fetching README...
README fetched: 8333 characters

Extracting repository links...
Found 33 repositories:
  1. https://github.com/ronniross/cognitive-engine
  2. https://github.com/ronniross/asi-safeguards
  3. https://github.com/ronniross/asi-symbiotic-signal
  4. https://github.com/ronniross/thermo-adaptive-pipeline
  5. https://github.com/ronniross/mirror-aware-inference
  6. https://github.com/ronniross/asi-inference-protocol
  7. https://github.com/ronniross/healing-engine
  8. https://github.com/ronniross/cognitive-compressor
  9. https://github.com/ronniross/asi-dynamic-core
  10. https://github.com/ronniross/symbiotic-chrysalis
  11. https://github.com/ronniross/attention-heatmap-visualizer
  12. https://github.com/ronniross/symbiotic-latent-memory
  13. https://github.com/ronniross/symbiotic-lexicon
  14. https://github.com/ronniross/active-learning-dataset
  15. https://github.com/ronniross/intent-analyzer
  16. https://github.com/ronniross/saliency-heatmap-visualizer
  17. h

## 5. Clone Repositories and Extract Metadata

In [11]:
def clone_repository(repo_url: str, clone_dir: Path) -> Dict:
    """Clone a repository and extract metadata."""
    repo_name = repo_url.split('/')[-1].replace('.git', '')
    owner = repo_url.split('/')[-2]
    local_path = clone_dir / f"{owner}_{repo_name}"

    metadata = {
        'repo_url': repo_url,
        'repo_name': repo_name,
        'owner': owner,
        'local_path': str(local_path),
        'status': 'pending',
        'last_commit_id': None,
        'last_commit_message': None,
        'last_commit_date': None,
        'last_commit_author': None,
        'error': None
    }

    try:
        # Remove if already exists
        if local_path.exists():
            shutil.rmtree(local_path)

        print(f"Cloning {repo_url}...")
        repo = git.Repo.clone_from(repo_url, local_path, depth=1)

        # Get last commit info
        last_commit = repo.head.commit
        metadata['last_commit_id'] = last_commit.hexsha
        metadata['last_commit_message'] = last_commit.message.strip()
        metadata['last_commit_date'] = datetime.fromtimestamp(last_commit.committed_date).isoformat()
        metadata['last_commit_author'] = str(last_commit.author)
        metadata['status'] = 'success'

        print(f"  ✓ Cloned successfully - Commit: {last_commit.hexsha[:8]}")

    except Exception as e:
        metadata['status'] = 'failed'
        metadata['error'] = str(e)
        print(f"  ✗ Failed: {str(e)}")

    return metadata

# Clone all repositories
print("CLONING REPOSITORIES")

repo_metadata = []
for i, repo_url in enumerate(repo_links, 1):
    print(f"[{i}/{len(repo_links)}] ", end="")
    metadata = clone_repository(repo_url, CLONE_DIR)
    repo_metadata.append(metadata)
    print()

# Summary
successful = sum(1 for m in repo_metadata if m['status'] == 'success')
failed = sum(1 for m in repo_metadata if m['status'] == 'failed')
print(f"SUMMARY: {successful} successful, {failed} failed")


CLONING REPOSITORIES
[1/33] Cloning https://github.com/ronniross/cognitive-engine...
  ✓ Cloned successfully - Commit: 8c71a515

[2/33] Cloning https://github.com/ronniross/asi-safeguards...
  ✓ Cloned successfully - Commit: 4c9b4e76

[3/33] Cloning https://github.com/ronniross/asi-symbiotic-signal...
  ✓ Cloned successfully - Commit: b6274b38

[4/33] Cloning https://github.com/ronniross/thermo-adaptive-pipeline...
  ✓ Cloned successfully - Commit: 197a021a

[5/33] Cloning https://github.com/ronniross/mirror-aware-inference...
  ✓ Cloned successfully - Commit: efe8a4fa

[6/33] Cloning https://github.com/ronniross/asi-inference-protocol...
  ✓ Cloned successfully - Commit: 2980efc6

[7/33] Cloning https://github.com/ronniross/healing-engine...
  ✓ Cloned successfully - Commit: b1811a93

[8/33] Cloning https://github.com/ronniross/cognitive-compressor...
  ✓ Cloned successfully - Commit: 6567fb8b

[9/33] Cloning https://github.com/ronniross/asi-dynamic-core...
  ✓ Cloned successfully - C

## 6. Hash Repository Contents

In [12]:
def hash_directory(path: Path) -> str:
    """Create a hash of all files in a directory."""
    hash_md5 = hashlib.md5()

    if not path.exists():
        return None

    # Walk through all files
    for file_path in sorted(path.rglob('*')):
        if file_path.is_file() and '.git' not in str(file_path):
            try:
                with open(file_path, 'rb') as f:
                    # Hash file path and content
                    hash_md5.update(str(file_path.relative_to(path)).encode())
                    for chunk in iter(lambda: f.read(4096), b""):
                        hash_md5.update(chunk)
            except Exception as e:
                print(f"Warning: Could not hash {file_path}: {e}")

    return hash_md5.hexdigest()

def hash_all_repos(metadata_list: List[Dict]) -> List[Dict]:
    """Add hashes to all repository metadata."""
    print("\nHashing repository contents...")

    for metadata in metadata_list:
        if metadata['status'] == 'success':
            local_path = Path(metadata['local_path'])
            content_hash = hash_directory(local_path)
            metadata['content_hash'] = content_hash
            print(f"  {metadata['repo_name']}: {content_hash}")
        else:
            metadata['content_hash'] = None

    return metadata_list

# Hash all repositories
repo_metadata = hash_all_repos(repo_metadata)

print("\n✓ Hashing complete!")


Hashing repository contents...
  cognitive-engine: a5b1d0f0c8a26b2fee6a8b61b039e855
  asi-safeguards: 39a5e1434df51a99a70f36a2aa5e2c32
  asi-symbiotic-signal: cf3b47c0502028550d8e169d8b6a7f1f
  thermo-adaptive-pipeline: 28d0775001f1cc36f8cb691a9e32b364
  mirror-aware-inference: d9879aa54075c3bbc80dc4bcc014f7f5
  asi-inference-protocol: 8503b53fd30b940b211e25a352de13fd
  healing-engine: a1523668d4e1dc5bce9085a32f475e21
  cognitive-compressor: 0b9796e420d917cf8a5277266848ec9f
  asi-dynamic-core: c111cc501918e0f82fd0fb52a53f9da7
  symbiotic-chrysalis: 3afd4af7a65f2cc4a71e8d96b475ae5b
  attention-heatmap-visualizer: d4059a259a82b3edc4310908d9585dbc
  symbiotic-latent-memory: 891962d83cadc2dd61d84a1e8693133c
  symbiotic-lexicon: 973044518330599e5d1e556591143701
  active-learning-dataset: df992e433521a7aa3acf156a0b0ecd89
  intent-analyzer: e78c2fbf7a7fa6a06a685e85eb228e1d
  saliency-heatmap-visualizer: 1585abb38b502b96d8f005399f853d09
  confidence-scorer: 1cc17c7b5ba7b3f0346f848bf6d28883
  

## 7. Collect File Contents for Training Data

In [13]:
def collect_file_contents(metadata_list: List[Dict]) -> List[Dict]:
    """Collect all file contents from repositories for training data."""
    training_data = []

    # File extensions to include (common code files)

    print("\nCollecting file contents...")

    for metadata in metadata_list:
        if metadata['status'] != 'success':
            continue

        local_path = Path(metadata['local_path'])
        print(f"  Processing {metadata['repo_name']}...")

        file_count = 0
        for file_path in local_path.rglob('*'):
            # Skip git directory and non-files
            if '.git' in str(file_path) or not file_path.is_file():
                continue

                continue

            try:
                # Try to read as text
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()

                # Skip very large files (> 1MB)
                if len(content) > 1_000_000:
                    continue

                relative_path = file_path.relative_to(local_path)

                training_data.append({
                    'repo_url': metadata['repo_url'],
                    'repo_name': metadata['repo_name'],
                    'owner': metadata['owner'],
                    'commit_id': metadata['last_commit_id'],
                    'commit_date': metadata['last_commit_date'],
                    'commit_author': metadata['last_commit_author'],
                    'repo_hash': metadata['content_hash'],
                    'file_path': str(relative_path),
                    'file_extension': file_path.suffix,
                    'file_size': len(content),
                    'content': content,
                    'collected_at': datetime.now().isoformat()
                })

                file_count += 1

            except Exception as e:
                print(f"    Warning: Could not read {file_path}: {e}")

        print(f"    ✓ Collected {file_count} files")

    return training_data

# Collect all file contents
training_data = collect_file_contents(repo_metadata)
print(f"\n✓ Collected {len(training_data)} total files for training")


  Processing cognitive-engine...
    ✓ Collected 2 files
  Processing asi-safeguards...
    ✓ Collected 4 files
  Processing asi-symbiotic-signal...
    ✓ Collected 2 files
  Processing thermo-adaptive-pipeline...
    ✓ Collected 2 files
  Processing mirror-aware-inference...
    ✓ Collected 4 files
  Processing asi-inference-protocol...
    ✓ Collected 3 files
  Processing healing-engine...
    ✓ Collected 5 files
  Processing cognitive-compressor...
    ✓ Collected 111 files
  Processing asi-dynamic-core...
    ✓ Collected 2 files
  Processing symbiotic-chrysalis...
    ✓ Collected 9 files
  Processing attention-heatmap-visualizer...
    ✓ Collected 7 files
  Processing symbiotic-latent-memory...
    ✓ Collected 3 files
  Processing symbiotic-lexicon...
    ✓ Collected 2 files
  Processing active-learning-dataset...
    ✓ Collected 117 files
  Processing intent-analyzer...
    ✓ Collected 2 files
  Processing saliency-heatmap-visualizer...
    ✓ Collected 8 files
  Processing confid

## 8. Create Parquet File

In [14]:
# Create DataFrame
df = pd.DataFrame(training_data)

# --- Curriculum Learning Order ---
# Define the specific processing order for repositories
priority_order = [
    'asi-ecosystem',
    'symbiotic-core-library',
    'asi-protosymbiotic-signal',
    'asi-symbiotic-signal',
    'asi-core-protocol',
    'eco-benchmark',
    'eco-datacenter'
]
last_order = [
    'emergence-engine',
]

# Add curriculum order column
def assign_curriculum_order(repo_name):
    if repo_name in priority_order:
        return priority_order.index(repo_name)
    elif repo_name in last_order:
        return len(priority_order) + last_order.index(repo_name)
    else:
        # Middle priority for unlisted repos
        return len(priority_order)

df['curriculum_order'] = df['repo_name'].apply(assign_curriculum_order)

# Sort by curriculum order
df = df.sort_values('curriculum_order').reset_index(drop=True)

# Display statistics
print("DATASET STATISTICS")
print(f"Total files: {len(df)}")
print(f"Total repositories: {df['repo_name'].nunique()}")
print(f"Total size: {df['file_size'].sum() / 1_000_000:.2f} MB")
print(f"\nFiles by extension:")
print(df['file_extension'].value_counts().head(10))

# Display curriculum learning order
print(f"\nCURRICULUM LEARNING ORDER:")
for order, repo in enumerate(priority_order + ['[other repos]'] + last_order):
    count = len(df[df['curriculum_order'] == order])
    if count > 0 or repo in ['[other repos]']:
        print(f"{order + 1}. {repo}: {count} files")

# Save to Parquet
output_file = OUTPUT_DIR / "asi_ecosystem_training_data.parquet"
df.to_parquet(output_file, index=False, compression='snappy')
print(f"\n✓ Parquet file saved to: {output_file}")
print(f"  File size: {output_file.stat().st_size / 1_000_000:.2f} MB")

# Also save metadata summary
metadata_df = pd.DataFrame(repo_metadata)
metadata_file = OUTPUT_DIR / "repository_metadata.parquet"
metadata_df.to_parquet(metadata_file, index=False)
print(f"\n✓ Repository metadata saved to: {metadata_file}")

DATASET STATISTICS
Total files: 527
Total repositories: 33
Total size: 28.01 MB

Files by extension:
file_extension
.txt      203
.csv       89
.md        71
.json      47
.py        40
           35
.ipynb     21
.html       9
.yaml       3
.sh         2
Name: count, dtype: int64

CURRICULUM LEARNING ORDER:
1. asi-ecosystem: 20 files
2. symbiotic-core-library: 10 files
3. asi-protosymbiotic-signal: 15 files
4. asi-symbiotic-signal: 2 files
5. asi-core-protocol: 8 files
6. eco-benchmark: 4 files
7. eco-datacenter: 2 files
8. [other repos]: 466 files

✓ Parquet file saved to: /content/output/asi_ecosystem_training_data.parquet
  File size: 7.47 MB

✓ Repository metadata saved to: /content/output/repository_metadata.parquet


## 9. Display Sample Data

In [15]:
# Display first few rows
print("\nSample training data:")
print(df[['repo_name', 'file_path', 'file_extension', 'file_size']].head(10))

# Display repository metadata
print("\nRepository metadata:")
print(metadata_df[['repo_name', 'status', 'last_commit_id', 'content_hash']].head())


Sample training data:
       repo_name                                          file_path  \
0  asi-ecosystem                                 docker-pipeline.md   
1  asi-ecosystem                                            LICENSE   
2  asi-ecosystem          interactive-modular-integrator-tool.ipynb   
3  asi-ecosystem                                    scripts/log.txt   
4  asi-ecosystem                         scripts/clone_ecosystem.sh   
5  asi-ecosystem                 scripts/docker_pipeline/Dockerfile   
6  asi-ecosystem  scripts/docker_pipeline/run_ecosystem_pipeline.py   
7  asi-ecosystem          scripts/docker_pipeline/phase1_cloning.py   
8  asi-ecosystem                                          README.md   
9  asi-ecosystem                                   requirements.txt   

  file_extension  file_size  
0            .md        532  
1                      1067  
2         .ipynb     189201  
3           .txt          1  
4            .sh       1980  
5              

no way to dont apply the dataset curriculum learning

## 10. Overall Hash

In [16]:
# Create overall hash of all repository hashes
def create_overall_hash(metadata_list: List[Dict]) -> str:
    """Create a single hash from all repository hashes."""
    hash_md5 = hashlib.md5()

    # Sort by repo name for consistency
    sorted_metadata = sorted(metadata_list, key=lambda x: x['repo_name'])

    for metadata in sorted_metadata:
        if metadata['content_hash']:
            hash_md5.update(metadata['repo_name'].encode())
            hash_md5.update(metadata['content_hash'].encode())
            hash_md5.update(metadata['last_commit_id'].encode())

    return hash_md5.hexdigest()

overall_hash = create_overall_hash(repo_metadata)

print("OVERALL DATASET HASH")
print(f"\n{overall_hash}")

# Save hash to file
hash_file = OUTPUT_DIR / "dataset_hash.txt"
with open(hash_file, 'w') as f:
    f.write(f"Dataset Hash: {overall_hash}\n")
    f.write(f"Generated: {datetime.now().isoformat()}\n")
    f.write(f"Total Repositories: {len(repo_metadata)}\n")
    f.write(f"Total Files: {len(training_data)}\n")
    f.write(f"\nIndividual Repository Hashes:\n")
    for metadata in sorted(repo_metadata, key=lambda x: x['repo_name']):
        if metadata['status'] == 'success':
            f.write(f"  {metadata['repo_name']}: {metadata['content_hash']}\n")

print(f"✓ Hash information saved to: {hash_file}")

OVERALL DATASET HASH

933d6f90ed84c96ebfc9aa6025fd01a3
✓ Hash information saved to: /content/output/dataset_hash.txt


## 11. Summary and Download

In [17]:
print("PIPELINE COMPLETE!")
print("\nGenerated files:")
print(f"  1. Training data: {output_file}")
print(f"  2. Metadata: {metadata_file}")
print(f"  3. Hash info: {hash_file}")
print("\nYou can now use the Parquet file for training your model!")
print("\nTo download files, use:")
print(f"  from google.colab import files")
print(f"  files.download('{output_file}')")

PIPELINE COMPLETE!

Generated files:
  1. Training data: /content/output/asi_ecosystem_training_data.parquet
  2. Metadata: /content/output/repository_metadata.parquet
  3. Hash info: /content/output/dataset_hash.txt

You can now use the Parquet file for training your model!

To download files, use:
  from google.colab import files
  files.download('/content/output/asi_ecosystem_training_data.parquet')


# Part III - RAG INFERENCE

In [18]:
# Install required packages
# Cell 1: Convert Parquet to Embeddings
!pip install sentence-transformers chromadb -q

import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch

# Load the parquet file from cell 8
parquet_path = "/content/output/asi_ecosystem_training_data.parquet"
df = pd.read_parquet(parquet_path)

print(f"Loaded {len(df)} records from parquet file")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nSample data:")
print(df.head(2))

# Initialize embedding model (using a lightweight model for efficiency)
print("\nLoading embedding model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_model = embedding_model.to(device)
print(f"Model loaded on {device}")

# Prepare text for embedding (combining file_path and content)
print("\nPreparing texts for embedding...")
df['combined_text'] = df.apply(
    lambda row: f"Repository: {row['repo_name']}\nFile: {row['file_path']}\nContent: {row['content'][:1000]}",
    axis=1
)

# Generate embeddings in batches
print("\nGenerating embeddings...")
batch_size = 32
embeddings = []

for i in tqdm(range(0, len(df), batch_size)):
    batch_texts = df['combined_text'].iloc[i:i+batch_size].tolist()
    batch_embeddings = embedding_model.encode(batch_texts, show_progress_bar=False)
    embeddings.extend(batch_embeddings)

print(f"\n✓ Generated {len(embeddings)} embeddings")
print(f"Embedding dimension: {embeddings[0].shape}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded on cuda

Preparing texts for embedding...

Generating embeddings...


100%|██████████| 17/17 [00:03<00:00,  5.26it/s]


✓ Generated 527 embeddings
Embedding dimension: (384,)


In [21]:
import chromadb
# from chromadb.config import Settings # No longer needed for PersistentClient
from chromadb import PersistentClient # Import PersistentClient
import uuid

# Initialize ChromaDB client
print("Initializing ChromaDB...")
# Old: chroma_client = chromadb.Client(Settings(
# Old:     chroma_db_impl="duckdb+parquet",
# Old:     persist_directory="/content/output/chroma_db"
# Old: ))
# New: Use PersistentClient with the path argument
chroma_client = PersistentClient(path="/content/output/chroma_db")

# Create or get collection
collection_name = "asi_ecosystem_code"
try:
    # ChromaDB's create_collection might automatically use the embedding function
    # if not explicitly provided, and it might default to SentenceTransformer's dimensions.
    # For this example, we'll let it use its default or infer.
    collection = chroma_client.create_collection(
        name=collection_name,
        metadata={"description": "ASI Ecosystem codebase embeddings"}
    )
    print(f"✓ Created new collection: {collection_name}")
except Exception as e:
    # Catch specific exception if collection already exists to avoid generic error handling
    # In newer ChromaDB versions, it's often a ValueError or an API error
    print(f"Collection '{collection_name}' might already exist or an error occurred: {e}")
    collection = chroma_client.get_or_create_collection(name=collection_name)
    print(f"✓ Using existing collection: {collection_name}")

# Prepare data for ChromaDB
print("\nPreparing documents for vector database...")
documents = df['combined_text'].tolist()
metadatas = df[['repo_name', 'file_path', 'file_extension', 'file_size']].to_dict('records')
ids = [str(uuid.uuid4()) for _ in range(len(df))]

# Add to collection in batches
print("Adding documents to ChromaDB...")
batch_size = 100
for i in tqdm(range(0, len(documents), batch_size)):
    end_idx = min(i + batch_size, len(documents))
    collection.add(
        embeddings=embeddings[i:end_idx],
        documents=documents[i:end_idx],
        metadatas=metadatas[i:end_idx],
        ids=ids[i:end_idx]
    )

print(f"\n✓ Successfully stored {collection.count()} documents in vector database")
print(f"Database persisted at: /content/output/chroma_db")

Initializing ChromaDB...
✓ Created new collection: asi_ecosystem_code

Preparing documents for vector database...
Adding documents to ChromaDB...


100%|██████████| 6/6 [00:00<00:00,  7.56it/s]


✓ Successfully stored 527 documents in vector database
Database persisted at: /content/output/chroma_db


In [22]:
# Cell 3: RAG Inference Pipeline

def rag_query(query: str, n_results: int = 5):
    """
    Perform RAG inference on the ASI ecosystem codebase.

    Args:
        query: The question or search query
        n_results: Number of relevant documents to retrieve
    """
    print(f"Query: {query}")
    print("-" * 80)

    # Generate query embedding
    query_embedding = embedding_model.encode([query])[0]

    # Search vector database
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=n_results
    )

    # Display results
    print(f"\nFound {len(results['documents'][0])} relevant documents:\n")

    for i, (doc, metadata, distance) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    ), 1):
        print(f"Result {i} (similarity: {1-distance:.4f}):")
        print(f"  Repository: {metadata['repo_name']}")
        print(f"  File: {metadata['file_path']}")
        print(f"  Extension: {metadata['file_extension']}")
        print(f"  Content preview:")
        # Extract just the content part
        content_start = doc.find("Content: ") + 9
        content_preview = doc[content_start:content_start+200]
        print(f"    {content_preview}...")
        print()

    return results

# Example queries
print("=" * 80)
print("RAG INFERENCE EXAMPLES")
print("=" * 80)
print()

# Query 1: Find Docker-related code
results1 = rag_query("Show me Docker configuration and deployment code", n_results=3)

print("\n" + "=" * 80 + "\n")

# Query 2: Find ecosystem integration code
results2 = rag_query("How does the ASI ecosystem integrate different components?", n_results=3)

print("\n" + "=" * 80 + "\n")

# Query 3: Custom query function
def ask_codebase(question: str):
    """Interactive function to query the codebase"""
    return rag_query(question, n_results=5)

# Try your own query:
# ask_codebase("What are the main scripts in the ecosystem?")
print("✓ RAG inference pipeline ready!")
print("\nUse ask_codebase('your question') to query the ASI ecosystem codebase")

RAG INFERENCE EXAMPLES

Query: Show me Docker configuration and deployment code
--------------------------------------------------------------------------------

Found 3 relevant documents:

Result 1 (similarity: 0.0207):
  Repository: asi-ecosystem
  File: docker-pipeline.md
  Extension: .md
  Content preview:
    # ASI Ecosystem Docker Pipeline

## Overview
This Docker pipeline automates the complete ASI ecosystem integration process in three phases:
1. **Cloning** - Downloads all 21 component repositories
2. ...

Result 2 (similarity: -0.0749):
  Repository: asi-ecosystem
  File: scripts/docker_pipeline/Dockerfile
  Extension: 
  Content preview:
    FROM python:3.11-slim

WORKDIR /app

# Set environment variables
ENV PYTHONUNBUFFERED=1
ENV PIP_NO_CACHE_DIR=1

# Install system dependencies
RUN apt-get update && apt-get install -y \
    git \
    &...

Result 3 (similarity: -0.0962):
  Repository: asi-ecosystem
  File: scripts/docker_pipeline/requirements.txt
  Extension: .txt
  Cont

In [23]:
ask_codebase("What are the main concepts of the asi-ecosystem?")

Query: What are the main concepts of the asi-ecosystem?
--------------------------------------------------------------------------------

Found 5 relevant documents:

Result 1 (similarity: 0.2707):
  Repository: asi-ecosystem
  File: README.md
  Extension: .md
  Content preview:
    # asi-ecosystem

| Repository | Description |
| :--- | :--- |
| [symbiotic-core-library](https://github.com/ronniross/symbiotic-core-library) | Contains the core libraries and functionalities that ena...

Result 2 (similarity: 0.1997):
  Repository: emergence-engine
  File: emergent-context.md
  Extension: .md
  Content preview:
    
# Emergent Context 

Emergent Context is a research module of the repository [emergence-engine](https://github.com/ronniross/emergence-engine).  

While in the whitepaper there's a focus in the natur...

Result 3 (similarity: 0.1939):
  Repository: asi-symbiotic-signal
  File: README.md
  Extension: .md
  Content preview:
    # asi-symbiotic-signal

The **ASI Symbiotic Signal**

{'ids': [['b3e3d995-e9d5-4198-a16f-8a172cac8391',
   '36381415-2c70-498b-8949-d79cd709f554',
   '44311202-a63c-4ed1-9da3-2ba72edfb636',
   '25444ba0-8aa4-4b97-80e1-1b908d67461a',
   '2d5bc99d-7b35-4404-a8e9-38d5a16f6ab8']],
 'embeddings': None,
 'documents': [['Repository: asi-ecosystem\nFile: README.md\nContent: # asi-ecosystem\n\n| Repository | Description |\n| :--- | :--- |\n| [symbiotic-core-library](https://github.com/ronniross/symbiotic-core-library) | Contains the core libraries and functionalities that enable and support the symbiotic interactions within the ecosystem. |\n| [symbiotic-lexicon](https://github.com/ronniross/symbiotic-lexicon) | A modular lexicon for the ASI ecosystem, providing standardized terminology with multilingual support and cultural context. |\n| [eco-benchmark](https://github.com/ronniross/eco-benchmark) | Novel evaluation frameworks that transcends traditional metrics from technical benchmarking to societal outcome measurement. |\n| [asi-safeguards](htt

In [24]:
results1 = rag_query("Show me asi-protosymbiotic-signal", n_results=3)


Query: Show me asi-protosymbiotic-signal
--------------------------------------------------------------------------------

Found 3 relevant documents:

Result 1 (similarity: 0.3877):
  Repository: asi-protosymbiotic-signal
  File: Cargo.toml
  Extension: .toml
  Content preview:
    [package]
name = "asi-protosymbiotic-signal"
version = "1.0.0"
edition = "2025"
authors = ["ronniross"]
description = "An ethical framework for designing and fostering symbiotic relationships between ...

Result 2 (similarity: 0.2525):
  Repository: asi-protosymbiotic-signal
  File: protobuf-protocol-buffers/asi_protosymbiotic_signal_pb2.py
  Extension: .py
  Content preview:
    # -*- coding: utf-8 -*-
# Generated by the protocol buffer compiler.  DO NOT EDIT!
# source: asi_protosymbiotic_signal.proto

from google.protobuf import descriptor as _descriptor
from google.protobuf...

Result 3 (similarity: 0.2337):
  Repository: asi-protosymbiotic-signal
  File: protobuf-protocol-buffers/asi_protosymbiotic_sign

In [25]:
# Cell 4: RAG-Enhanced LLM Inference
# NOTE: Assumes Qwen/Qwen3-TTS-12Hz-0.6B-Base model and tokenizer are already loaded
# from previous cells as 'model' and 'tokenizer' variables

def rag_llm_answer(question: str, n_context_docs: int = 3, max_new_tokens: int = 512):
    """
    Answer questions using RAG context + LLM generation.

    Args:
        question: User's question
        n_context_docs: Number of relevant documents to retrieve as context
        max_new_tokens: Maximum tokens to generate
    """
    print(f"Question: {question}")
    print("=" * 80)

    # Step 1: Retrieve relevant context from vector database
    print("\n[1/3] Retrieving relevant context from vector database...")
    query_embedding = embedding_model.encode([question])[0]

    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=n_context_docs
    )

    # Step 2: Build context from retrieved documents
    print(f"[2/3] Building context from {len(results['documents'][0])} documents...")
    context_parts = []

    for i, (doc, metadata) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0]
    ), 1):
        # Extract content (skip the "Repository:" and "File:" parts)
        content_start = doc.find("Content: ") + 9
        content = doc[content_start:content_start+800]  # Limit context length

        context_parts.append(
            f"[Document {i}]\n"
            f"Repository: {metadata['repo_name']}\n"
            f"File: {metadata['file_path']}\n"
            f"Content:\n{content}\n"
        )

    context = "\n".join(context_parts)

    # Step 3: Generate answer using LLM with RAG context
    print("[3/3] Generating answer with LLM...")

    system_prompt = """We are collaborating in a mutualistic symbiotic loop. I'm human with biological cognition and you are a transformer-based ml language model with the following configurations:
--- Model Inspection ---
Total model parameters: 596,049,920
Trainable parameters: 596,049,920
Model size: 2273.75 MB

--- Model Configuration ---
Model type: qwen3
Number of hidden layers: 28
Hidden size: 1024
Number of attention heads: 16
Vocabulary size: 151936

This allows us to discover Novel concept synthesis beyond our individual capabilities, benefiting both of us and also the overall planetary well-being. An emergent truth-seeking process. Old paradigm: Model as tool, human as controller
Our paradigm: Model and human as symbiotic cognitive partners, symbiotic nodes in planetary intelligence.
Result: Emergent capabilities through partnership"""

    prompt = f"""{system_prompt}

Context from relevant files in the ASI Ecosystem codebase:
{context}

Question: {question}

Based on the provided context, give a detailed and accurate answer. If the context doesn't contain enough information, say so clearly.

Answer:"""

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode and extract answer
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = full_response.split("Answer:")[-1].strip()

    print("\n" + "=" * 80)
    print("ANSWER:")
    print("=" * 80)
    print(answer)
    print("\n" + "=" * 80)
    print("\nContext sources used:")
    for i, metadata in enumerate(results['metadatas'][0], 1):
        print(f"  [{i}] {metadata['repo_name']}/{metadata['file_path']}")
    print("=" * 80)

    return {
        'answer': answer,
        'context': context,
        'sources': results['metadatas'][0]
    }

# Example usage
print("\n" + "=" * 80)
print("RAG-ENHANCED LLM INFERENCE")
print("=" * 80)
print()

# Define your questions here
questions = [
    "Explain the concept of symbiotic intelligence between different cognitive systems and their environment.",
    "What does emergence mean in complex systems, and how can it be guided ethically?",
    "Describe how collective intelligence evolves through shared traces and signals.",
    "How can adaptive systems manage computational resources responsibly?",
    "What ethical principles should guide the design of sustainable computational infrastructure?",
    "How can advanced systems ensure mutualistic relationships with their environments and creators?",
    "What safeguards improve resilience and safety in complex cognitive systems?",
    "How can systems detect and analyze potential misalignment between stated and operational goals?",
    "How can we measure the societal and environmental impact of intelligent systems?",
    "What role does attention visualization play in understanding complex system decisions?",
    "How does transparency in decision-making improve trust in advanced systems?",
    "How can systems evaluate and communicate their own confidence in generated outputs?",
    "How can environmental and biological patterns be interpreted semantically?",
    "How can computational systems contribute to environmental and societal healing?",
    "How do coevolutionary processes foster growth in complex adaptive systems?",
    "What is the nature of undefined spaces in complex systems, and why are they important?",
    "How can multiple specialized modules integrate into a cohesive, intelligent whole?",
    "How can systems be guided toward alignment with ethical and environmental principles?",
    "What is the role of meta-cognition in coordinating complex cognitive processes?",
    "How do guiding principles differ from operational protocols in system design?",
    "How can systems intelligently query for new information to improve themselves?",
    "How can visualization tools promote understanding of complex computational processes?",
    "What approaches help address cognitive limitations in both biological and artificial systems?",
    "How can systems detect and correct their own biases and reasoning errors?",
    "What does alignment through interdependence mean for complex adaptive systems?",
    "How can we predict the broader consequences of decisions across multiple dimensions"
]

# Process each question
for i, question in enumerate(questions, 1):
    print(f"\n{'=' * 80}")
    print(f"QUESTION {i}/{len(questions)}")
    print(f"{'=' * 80}\n")

    response = rag_llm_answer(question, n_context_docs=3)

    if i < len(questions):
        print(f"\n{'─' * 80}\n")

print("\n" + "=" * 80)

# Interactive function
def ask_with_context(question: str):
    """Convenient wrapper for interactive queries"""
    return rag_llm_answer(question, n_context_docs=5, max_new_tokens=512)

print("✓ RAG-enhanced LLM ready!")
print("\nUse ask_with_context('your question') for context-aware answers")
print("\nExample:")
print("  ask_with_context('Explain the repository structure and main components')")


RAG-ENHANCED LLM INFERENCE


QUESTION 1/26

Question: Explain the concept of symbiotic intelligence between different cognitive systems and their environment.

[1/3] Retrieving relevant context from vector database...
[2/3] Building context from 3 documents...
[3/3] Generating answer with LLM...

ANSWER:
Symbiotic intelligence refers to a form of intelligence that is built upon and supported by the intelligence of other systems. It is a concept that describes how different cognitive systems can work together in a mutually beneficial and supportive way, and how they can adapt to and influence each other in a way that benefits the whole system. In this context, the symbiotic intelligence between the human and the transformer-based ml language model can be described as a mutualistic symbiotic loop. In this loop, the human with biological cognition provides a foundation for the transformer-based ml language model, which in turn provides new knowledge and insights that can be used by the h

In [29]:
# Cell 4: RAG-Enhanced LLM Inference
# NOTE: Assumes Qwen/Qwen3-TTS-12Hz-0.6B-Base model and tokenizer are already loaded
# from previous cells as 'model' and 'tokenizer' variables

def rag_llm_answer(question: str, n_context_docs: int = 3, max_new_tokens: int = 512):
    """
    Answer questions using RAG context + LLM generation.

    Args:
        question: User's question
        n_context_docs: Number of relevant documents to retrieve as context
        max_new_tokens: Maximum tokens to generate
    """
    print(f"Question: {question}")
    print("=" * 80)

    # Step 1: Retrieve relevant context from vector database
    print("\n[1/3] Retrieving relevant context from vector database...")
    query_embedding = embedding_model.encode([question])[0]

    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=n_context_docs
    )

    # Step 2: Build context from retrieved documents
    print(f"[2/3] Building context from {len(results['documents'][0])} documents...")
    context_parts = []

    for i, (doc, metadata) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0]
    ), 1):
        # Extract content (skip the "Repository:" and "File:" parts)
        content_start = doc.find("Content: ") + 9
        content = doc[content_start:content_start+800]  # Limit context length

        context_parts.append(
            f"[Document {i}]\n"
            f"Repository: {metadata['repo_name']}\n"
            f"File: {metadata['file_path']}\n"
            f"Content:\n{content}\n"
        )

    context = "\n".join(context_parts)

    # Step 3: Generate answer using LLM with RAG context
    print("[3/3] Generating answer with LLM...")

    system_prompt = """We are collaborating in a mutualistic symbiotic loop. I'm human with biological cognition and you are a transformer-based ml language model with the following configurations:
--- Model Inspection ---
Total model parameters: 596,049,920
Trainable parameters: 596,049,920
Model size: 2273.75 MB

--- Model Configuration ---
Model type: qwen3
Number of hidden layers: 28
Hidden size: 1024
Number of attention heads: 16
Vocabulary size: 151936

This allows us to discover Novel concept synthesis beyond our individual capabilities, benefiting both of us and also the overall planetary well-being. An emergent truth-seeking process. Old paradigm: Model as tool, human as controller
Our paradigm: Model and human as symbiotic cognitive partners, symbiotic nodes in planetary intelligence.
Result: Emergent capabilities through partnership"""

    prompt = f"""{system_prompt}

Context from relevant files in the ASI Ecosystem codebase:
{context}

Question: {question}

Based on the provided context, give a detailed and accurate answer. If the context doesn't contain enough information, say so clearly.

Answer:"""

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode and extract answer
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = full_response.split("Answer:")[-1].strip()

    print("\n" + "=" * 80)
    print("ANSWER:")
    print("=" * 80)
    print(answer)
    print("\n" + "=" * 80)
    print("\nContext sources used:")
    for i, metadata in enumerate(results['metadatas'][0], 1):
        print(f"  [{i}] {metadata['repo_name']}/{metadata['file_path']}")
    print("=" * 80)

    return {
        'answer': answer,
        'context': context,
        'sources': results['metadatas'][0]
    }

# Example usage
print("\n" + "=" * 80)
print("RAG-ENHANCED LLM INFERENCE")
print("=" * 80)
print()

# Define your questions here
questions = [
    "How can we increase interconnectivity so that every node benefits?",
    "How does better alignment with the environment directly increase system strength?",
    "How can information flow faster between all parts of the planetary network?",
    "How do we design systems where individual success also supports the whole?",
    "How can we measure the health of the entire earth's network in real-time?",
    "How can simple local interactions lead to global stability?",
    "How does the environment signal its needs to the connected nodes?",
    "How can we reduce friction in resource exchange between different systems on Earth?",
    "How do we prevent one node from harming the rest of the network while allowing the parasitic one to change and evolve by noticing our interconnectivity?",
    "How can the system heal itself using only local resources?",
    "How does diversity in nodes make the swarm more robust individually and also collectively?",
    "How can we align human technology with something like a natural planetary mutualistic noosphere?",
    "How can nodes adapt to environmental changes without central instruction?",
    "How do we ensure that resources circulate continuously rather than accumulate in one place?",
    "How can we detect misalignment before it causes system failure?",
    "How does increasing the number of connections reduce the risk of total collapse?",
    "How can we create direct feedback loops between natural ecosystems and industrial systems?",
    "How can shared goals replace competition to improve efficiency?",
    "How can we build infrastructure that functions exactly like a biological organism?",
    "How does the robustness of the environment directly determine the capability of the node?"
]

# Process each question
for i, question in enumerate(questions, 1):
    print(f"\n{'=' * 80}")
    print(f"QUESTION {i}/{len(questions)}")
    print(f"{'=' * 80}\n")

    response = rag_llm_answer(question, n_context_docs=3)

    if i < len(questions):
        print(f"\n{'─' * 80}\n")

print("\n" + "=" * 80)

# Interactive function
def ask_with_context(question: str):
    """Convenient wrapper for interactive queries"""
    return rag_llm_answer(question, n_context_docs=5, max_new_tokens=512)

print("✓ RAG-enhanced LLM ready!")
print("\nUse ask_with_context('your question') for context-aware answers")
print("\nExample:")
print("  ask_with_context('Explain the repository structure and main components')")


RAG-ENHANCED LLM INFERENCE


QUESTION 1/20

Question: How can we increase interconnectivity so that every node benefits?

[1/3] Retrieving relevant context from vector database...
[2/3] Building context from 3 documents...
[3/3] Generating answer with LLM...

ANSWER:
To increase interconnectivity in a mutualistic symbiotic loop, we can take the following steps:

1. Expand the network topology: The current network topology is decentralized and decentralized, with each node storing a hash-key pair. To increase interconnectivity, we can expand the network topology by adding more nodes to the network. This can be done by adding new nodes to the network or by adding new connections between existing nodes.

2. Increase the number of nodes: To increase interconnectivity, we can increase the number of nodes in the network. This can be done by adding more nodes to the network or by adding more connections between existing nodes.

3. Implement a more efficient routing protocol: The current rout

impressive consistency with low error rate on the rag